# Harmonizing a test set of FreeSurfer 7.1.1 volumes using NeuroHarmony (COMBAT + MRI-QC)

## E-DADS

### Steps

1. Load FS and MRIQC outputs
2. Wrangle to match `dummy_test.csv` header and divide volumes by `ICV`/`eTIV`
3. Load trained model
4. Run it

### Observations

- Does it work only for cross-sectional test data?

### Authors

- Neil Oxtoby, UCL: step 2 (wrangling)
- Vikram Venkatraghavan, AUMC (everything else)

In [ ]:
import pickle 
import pandas as pd 
import numpy as np 
from neuroharmony import Neuroharmony
import warnings 

warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path

## EDIT ME: Paths to data, CSV filenames, etc.

In [ ]:
# CHANGE ME
edads_folder = Path.home() / "Documents" / "research-projects" / "E-DADS_MR-T046422-1"
top_folder = edads_folder / "projects" / "2023-05_Harmonization_Paper" / "20231023-Model_For_Harmonizing_AIBL"

csv_aibl_demographics = top_folder / 'AIBLMERGE_freesurfer7p1p1-20220915.csv'
csv_test_freesurfer7p1p1 = top_folder / 'aibl_freesurfer7p1p1_frompierrick.csv'
tsv_test_mriqc0p16p1     = top_folder / 'aibl_mriqc_from_pierrick.tsv'

csv_test_wrangled   = top_folder / 'aibl_test_for_neuroharmony.csv'
csv_test_harmonised = Path(str(csv_test_wrangled).replace('.csv','-harmonised.csv'))

str_pickle = top_folder / 'pickle_files'

if 'noxtoby' in str(Path().home()):
    # Neil's path
    str_pickle = Path.home() / "Library/CloudStorage/OneDrive-UniversityCollegeLondon/Data-from-Neil-Oxtoby/E-DADS_harmonization/raw"


## Dummy CSV for correct formatting

In [ ]:
csv_dummy_test = top_folder / 'dummy_test.csv'
df_dummy_test = pd.read_csv(csv_dummy_test)

## CHECK ME: Load data

- Columns may need renaming
- Coding may need checking:
  - `Sex`: {0:'Male',1:'Female'}
  - `Diagnosis`: 0 unimpaired; 1 impaired
  - `scanner`: TBD
  - `cohort`: Just give it a name

In [ ]:
key = ['ID','visit'] # for merging tables

In [ ]:
df_test_mriqc = pd.read_csv(tsv_test_mriqc0p16p1,sep='\t')

df_test_fs    = pd.read_csv(csv_test_freesurfer7p1p1)
# Rename columns
df_test_fs.rename(columns={'rid':'ID'},inplace=True)

In [ ]:
df_aibl_demog = pd.read_csv(csv_aibl_demographics)
# Rename column(s)
df_aibl_demog.rename(columns={'TP':'visit'},inplace=True)
# Recode
df_aibl_demog['Sex_text'] = df_aibl_demog['Sex'].values
df_aibl_demog['Sex'] = df_aibl_demog['Sex'].map({"M":0,"F":1})
df_aibl_demog['Diagnosis'] = df_aibl_demog['Diagnostic Criteria'].map({'CN':0,'MCI':1,'AD':1})

# Add `cohort` column
df_aibl_demog['cohort'] = 'AIBL-LONI'

## Have a quick squiz at the data

In [ ]:
df_aibl_demog.drop_duplicates(subset=key,inplace=True) # Why are there always duplicates?
df_aibl_demog.head()

In [ ]:
df_dummy_test.head()

In [ ]:
df_test_fs.head()

In [ ]:
df_test_mriqc.head()

## EDIT ME: Data wrangling

This is likely to be specific to your CSVs. What I did:

- MRIQC
  - Added "ID" and "visit", based on the available BIDS name (sub/ses). For joining with FreeSurfer.
- FreeSurfer
  - Removed duplicates (subset=['ID','visit']), preferring those with fewer `SurfaceHoles`

In [ ]:
# MRIQC
print("Wrangling MRIQC\n")

# Add ID 
df_test_mriqc['ID'] = df_test_mriqc['bids_name'].map(lambda x: x.split('_ses')[0].split('sub-')[-1]).astype(int)
# Add visit
df_test_mriqc['visit'] = df_test_mriqc['bids_name'].map(lambda x: x.split('_ses-')[-1].split('_T1')[0])
# Reorder columns
df_test_mriqc = df_test_mriqc[key + [c for c in df_test_mriqc.columns.tolist() if c not in key]]
df_test_mriqc.head()

In [ ]:
# FreeSurfer
print("Identifying duplicates in FreeSurfer CSV\n")

print(f"Not sure why there are {key} duplicates in FreeSurfer, but they tend to differ by SurfaceHoles so perhaps that's useful:\n")
idx = list(df_test_fs.drop_duplicates(subset=key).index.values)
list(df_test_fs.index.values)
idx_having_duplicates = [ i for i in list(df_test_fs.index.values) if i not in idx ]
df_test_fs.loc[df_test_fs['ID'].isin(df_test_fs.loc[idx_having_duplicates,'ID'].tolist())].sort_values(by=key)

In [ ]:
# FreeSurfer
print("Wrangling FreeSurfer\n")

print(f"Before: df.shape: {df_test_fs.shape}")
# Drop duplicates
df_test_fs = df_test_fs.sort_values(by=key+['SurfaceHoles']).drop_duplicates(subset=key).reset_index(drop=True)
print(f"After:  df.shape: {df_test_fs.shape}")

## Merge FreeSurfer and MRIQC

Then match columns with `dummy_test.csv`

- Extract missing ones from wherever I can

In [ ]:
df_test = pd.merge(df_test_fs,df_test_mriqc,on=key,how='inner')

In [ ]:
# print(f"df_test_fs.shape   : {df_test_fs.shape}")
# print(f"df_test_fs.drop_duplicates().shape   : {df_test_fs.drop_duplicates(subset=key).shape}")
# print("")
# print(f"df_test_mriqc.shape: {df_test_mriqc.shape}")
# print(f"df_test_mriqc.drop_duplicates().shape   : {df_test_mriqc.drop_duplicates(subset=key+['bids_name']).shape}")
# print("")
# print(f"df_test.shape      : {df_test.shape}")
# print(f"df_test.drop_duplicates().shape      : {df_test.drop_duplicates(subset=key).shape}")

## Match test data columns to dummy

In [ ]:
# Harmonise the columns
in_both_dummy_and_test = [c for c in df_test.columns.tolist() if c in df_dummy_test.columns.tolist()]
in_test_and_not_dummy  = [c for c in df_test.columns.tolist() if c not in df_dummy_test.columns.tolist()]
in_dummy_and_not_test  = [c for c in df_dummy_test.columns.tolist() if c not in df_test.columns.tolist()]

# print(f'In both dummy and real:\n{", ".join(in_both_dummy_and_test)}\n')
# print("")

print(f'In dummy but not real (add these from AIBL):\n{", ".join(in_dummy_and_not_test)}\n')

# print("")
# print(f'In real  but not dummy:\n{", ".join(in_test_and_not_dummy)}')

In [ ]:
in_demog = ['cohort','Age','Sex','Diagnosis']
df_aibl_demog[in_demog]

In [ ]:
print(f"Demographics merge.\nBefore: df.shape: {df_test.shape}")
df_test = pd.merge(df_test,df_aibl_demog[key + in_demog],on=key,how='inner')
print(f"After:  df.shape: {df_test.shape}")

## FIXME: get `scanner` information from somewhere

In [ ]:
df_test['scanner'] = 'Unknown'

In [ ]:
colz = df_dummy_test.columns.tolist()
df_test = df_test[colz].copy()

## Wrangling: dividing test volumes by TIV

In [ ]:
# df_test.columns.tolist()

In [ ]:
vols = [c for c in df_test.columns.tolist() if ( 'vol' in str.lower(c) ) | ( 'left' in str.lower(c) ) | ( 'right' in str.lower(c) ) | (c=='CSF')]
tiv_col = 'eTIV'
for v in vols:
    df_test[v] = df_test[v]/df_test[tiv_col]


## Write out to CSV

In [ ]:
df_test.to_csv(csv_test_wrangled,index=False)
print(f"Wrangled test data written out to {csv_test_wrangled.name}")

<hr/>

<hr/>

<hr/>


## Load the wrangled test data

In [ ]:
Xtest = pd.read_csv(csv_test_wrangled, low_memory=False)
# Xtest needs to be in the format mentioned in dummy_test.csv 
# Note: The volumes should be expressed as fractions of eTIV
# Sex --> should be 0 / 1 {0 for male, 1 for female}
# Diagnosis --> should be 0 / 1 {0 for unimpaired cognition, 1 otherwise}


<hr/>

## Vikram's code from here

Contains some magic parameters based on the prescribed format for TEST data



In [ ]:
rois = list(Xtest)[2:88]
regression_features = list(Xtest)[88:157]

# Neil code
testing_on_single_roi = True 
if testing_on_single_roi:
    #rois = [rois[0]]
    rois = ['Left-Thalamus']
# END Neil code

covariates = ["Sex", "scanner", 'Age','Diagnosis']
eliminate_variance = ["scanner"]
discrete_cols = ['Sex','Diagnosis']
continuous_cols = ['Age']

Xtest_prepared = Xtest[['ID','cohort']+rois+regression_features+covariates].copy(deep=True)
Xharmonized = Xtest_prepared.copy(deep=True)

from tqdm import tqdm

for r in tqdm(rois):
    model = Neuroharmony(
        [r],
        regression_features,
        covariates,
        eliminate_variance,
        discrete_cols = discrete_cols,
        continuous_cols = continuous_cols,
        param_distributions=dict(
            RandomForestRegressor__n_estimators=[100, 200, 500],
            RandomForestRegressor__random_state=[42, 78],
            RandomForestRegressor__warm_start=[False, True],
        ),
        estimator_args=dict(n_jobs=64, random_state=42),
        randomized_search_args=dict(cv=5,n_jobs=64)
    )

    pickle_name = str_pickle / (r + '.pickle') # Edited by Neil

    # Added by Neil
    if not pickle_name.exists():
        print(f"{pickle_name.name} file not found. Looking for zip file.")
        pickle_name_zip = Path(str(pickle_name) + '.gz')
        if pickle_name_zip.exists():
            raise NotImplementedError(f"{pickle_name_zip.name} zip-file found. Extracting (FIXME: not implemented).")
        else:
            print(f"{pickle_name_zip.name} zip-file not found.")

    [model_roi,coverage_roi] = pickle.load(open(pickle_name,'rb'))

    model.models_by_feature_ = dict.fromkeys([r],model_roi)
    model.coverage_ = coverage_roi
    model.extra_vars = regression_features+covariates
    Xp=model.predict(Xtest_prepared)
    Xharmonized[r] = Xp[r].copy(deep=True)
    del model


In [ ]:
if testing_on_single_roi:
    from matplotlib import pyplot as plt
    #import seaborn
    fig,ax=plt.subplots(1,2,figsize=(12,5))
    ax[0].boxplot([Xtest_prepared[r],Xharmonized[r]])
    ax[0].set_xticks([1,2],labels=['Pre','Post'])
    ax[1].plot(Xtest_prepared[r],Xharmonized[r],'x')
    ax[1].set_xlabel('Pre')
    ax[1].set_ylabel('Post')
    fig.suptitle(r)
    fig.show()

# Write to file

In [ ]:
Xharmonized.to_csv(csv_test_harmonised, index=False)